# Chapter 10. Proccesing text
## Notebook for R

### 10.1 Reading and cleaning text

In [3]:
library(magrittr)
my_wiki_text = "<p><b>Communication</b> (from Latin <i>communicare</i>, meaning to share)"
print(my_wiki_text)

my_wiki_text %<>% gsub("<p>", " ", .) %>% gsub("<b>", " ", .) %>% gsub("</b>", " ", .) %>% gsub("<i>", " ", .) %>% gsub("</i>", " ", .) 
my_wiki_text_2 = gsub('[[:punct:]]','',my_wiki_text) #Remove punctuation
my_wiki_text_2 = tolower(my_wiki_text_2) #Convert to lower case
my_wiki_text_2 = trimws(gsub("\\s+", " ", my_wiki_text_2)) #Remove double spaces 
print(my_wiki_text_2)

[1] "<p><b>Communication</b> (from Latin <i>communicare</i>, meaning to share)"
[1] "communication from latin communicare meaning to share"


In [7]:
library(stringi)
my_wiki_text_3 = stri_replace_all(my_wiki_text, "", regex = "<.+?>")
my_wiki_text_3 = stri_trim(my_wiki_text_3)
print(my_wiki_text_3)

[1] "Communication  (from Latin  communicare , meaning to share)"


In [39]:
library(magrittr)
my_wiki_text = "<p><b>Communication</b> (from Latin <i>communicare</i>, meaning \"to share\")"
print(my_wiki_text)

#my_wiki_text = gsub("<p> |<b>", "", my_wiki_text) #remove HTML tags

'''
my_wiki_text %<>%
  gsub("<p>", "", .) %>%
  gsub("<b>", "", .) 




my_wiki_text = gsub("<b>", "", my_wiki_text) 
my_wiki_text = gsub("</b>", "", my_wiki_text) 
my_wiki_text = gsub("<i>", "", my_wiki_text) 
my_wiki_text = gsub("</i>", "", my_wiki_text) 
'''
#gsub(", |: |\\. |!","",A)

print(my_wiki_text)

ERROR: Error in parse(text = x, srcfile = src): <text>:7:3: unexpected string constant
18: my_wiki_text = gsub("</i>", "", my_wiki_text) 
19: '
      ^


In [ ]:
my_wiki_text = my_wiki_text.replace('<p>',' ').replace('<b>',' ').replace('</b>',' ').replace('<i>',' ').replace('</i>',' ') #remove HTML tags
my_wiki_text = "".join([l for l in my_wiki_text if l not in string.punctuation]) #Remove punctuation
my_wiki_text = my_wiki_text.lower()  #Convert to lower case
my_wiki_text = " ".join(my_wiki_text.split()) #Remove double spaces 

In [ ]:
Pattern
test  <- "junk junk<a href=\"/wiki/abstraction_(mathematics)\" title=\"abstraction (mathematics)\"> junk junk"
gsub(pattern = "<.*>", replacement = "", x = test)
[1] "junk junk junk junk"


gsub(pattern="[:punct:]", x=test, replacement=" ")